In [ ]:
import matplotlib.pyplot as plt
from keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional, Dropout
from keras.models import Model
from keras.activations import relu, sigmoid, softmax
import keras.backend as K
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import keras
import numpy as np
from IPython.display import display, Image

In [ ]:
input_layer = Input(shape=(32,128,1))
normalized_inputs = Lambda(lambda x: x / 255) (input_layer)

conv_layer1 = Conv2D(16, (3,3), activation = 'relu', kernel_initializer='he_normal' ,padding='same')(normalized_inputs)
conv_layer1 = Dropout(0.25)(conv_layer1)
conv_layer1 = Conv2D(32, (3,3), activation = 'relu', kernel_initializer='he_normal' ,padding='same')(conv_layer1)
pool_layer1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_layer1)

conv_layer2 = Conv2D(32, (3,3), activation = 'relu',kernel_initializer='he_normal' , padding='same')(pool_layer1)
conv_layer2 = BatchNormalization(axis=-1)(conv_layer2)
conv_layer2 = Dropout(0.25)(conv_layer2)
conv_layer2 = Conv2D(32, (3,3), activation = 'relu', kernel_initializer='he_normal' ,padding='same')(conv_layer2)
pool_layer2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_layer2)

conv_layer3 = Conv2D(32, (3,3), activation = 'relu', kernel_initializer='he_normal' ,padding='same')(pool_layer2)
conv_layer3 = BatchNormalization(axis=-1)(conv_layer3)
conv_layer3 = Dropout(0.25)(conv_layer3)
conv_layer3 = Conv2D(32, (3,3), activation = 'relu', kernel_initializer='he_normal' ,padding='same')(conv_layer3)
conv_layer4 = Conv2D(64, (3,3), activation = 'relu', kernel_initializer='he_normal' ,padding='same')(conv_layer3)
pool_layer4 = MaxPool2D(pool_size=(2, 1))(conv_layer4)

conv_layer5 = Conv2D(256, (3,3), activation = 'relu',kernel_initializer='he_normal' , padding='same')(pool_layer4)
batch_norm_layer5 = BatchNormalization()(conv_layer5)

conv_layer6 = Conv2D(256, (3,3), activation = 'relu',kernel_initializer='he_normal' , padding='same')(batch_norm_layer5)
batch_norm_layer6 = BatchNormalization()(conv_layer6)
pool_layer6 = MaxPool2D(pool_size=(2, 1))(batch_norm_layer6)

conv_layer7 = Conv2D(512, (2,2), activation = 'relu')(pool_layer6)

squeezed_layer = Lambda(lambda x: K.squeeze(x, 1))(conv_layer7)

blstm_layer1 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(squeezed_layer)
blstm_layer2 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(blstm_layer1)

output_layer = Dense(62+1, activation = 'softmax')(blstm_layer2)

# model to be used at test time
model = Model(input_layer, output_layer)

model.summary()

In [ ]:
def display_image_and_prediction(x,y):

  # load the saved best model weights
  model.load_weights(r'C:\Users\twalunjk\Downloads\DL\save\best_model.hdf5')
  prediction = model.predict(x.reshape(1,32,128,1))

  # use CTC decoder
  out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                         greedy=True)[0][0])
  x = x.reshape(32,128)
  plt.title('Input Image')
  plt.imshow(x)
  plt.axis('off')
  plt.show()

  # see the results
  for x in out:
      print("predicted text = ", end = '')
      for p in x:
          if int(p) != -1:
              print(char_list[int(p)], end = '')
      print('\n')

for _ in range(8):
  i = np.random.randint(300000)
  display_image_and_prediction(training_img[i],train_padded_txt[i])